<a href="https://colab.research.google.com/github/ValeriaZhivalovich/Tarot_bot/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Подключение диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from telebot import TeleBot, types
from collections import defaultdict
import re

# Хранилище выбора пользователя (сфера и карта)
user_selection = defaultdict(dict)

# API токен из BotFather
API_TOKEN = '8188103292:AAE77CaCABopAANA3yuXID-i5IGj6SkxBGc'
bot = TeleBot(API_TOKEN)

# Попытка загрузить файл с описанием значений карт
try:
    meaning_df = pd.read_csv('/content/drive/MyDrive/TaroBot/meaning.csv')
    print("Файл успешно загружен.")
except Exception as e:
    print(f"Ошибка при загрузке файла: {e}")

cards_dict = {
    "Шут": ["шут", "дурак", "крокодил", "путешественник", "фат", "глупец", "суфий", "душа", "безумец", "дурень", "сумасшедший", "арлекин", "дух эфира", "дух в поисках познания"],
    "Маг": ["маг", "скоморох", "шарлатан", "волшебник", "фокусник", "мастер", "жонглер", "фигляр", "существование", "решающий", "магия власти", "апостол", "посланник"],
    "Жрица": ["верховная жрица","жрица", "исида", "папесса", "врата святилища", "врата храма", "берегиня", "волшебница", "знахарка", "интуиция", "охраняющая", "внутренний голос", "монахиня", "ведьма", "жрица серебряной звезды", "ясность прозрения"],
    "Императрица": ["императрица", "хозяйка", "исида-урания", "изис-урания", "дочь всемогущества", "природа", "творчество", "рождение", "начало", "определяющий форму", "василиса", "царица", "веденица"],
    "Император": ["император", "хозяин", "кубический камень", "хранитель", "господин", "государь", "глава"],
    "Иерофант ": ["жрец", "первосвященник", "верховный жрец", "иерофант", "папа", "человек-аркан", "хозяин арканов", "маг вечности", "вспоминающий", "друид", "мастер", "духовник"],
    "Влюбленные": ["влюбленные", "два пути", "возлюбленный", "любовники", "дети гласа божьего", "выбор", "перекресток", "ересь"],
    "Колесница": ["колесница", "повозка", "колесничий", "перевоплощающийся", "осознанность", "колесница озириса", "вождь", "жизнь"],
    "Сила": ["сила", "лев", "вожделение", "воин", "храбрость", "наделяющий властью", "дочь пламенного меча", "сила духа", "лютый"],
    "Отшельник": ["отшельник", "гермит", "закрытый светильник", "инок", "маг голоса света", "советник", "шаман", "одиночество"],
    "Колесо Фортуны": ["колесо фортуны", "сфинкс", "колесо судьбы", "колесо сансары", "ткущий нить", "повелитель жизненных сил", "паутина", "изменение"],
    "Справедливость": ["справедливость", "фемида", "темис", "закон", "уравновешивающий", "дочь повелителя истины", "равновесие", "прорыв"],
    "Повешенный": ["повешенный", "жертва", "мученик", "дух всемогущих вод", "новое видение", "преданный", "мессия", "милосердие", "зодиак", "висельник"],
    "Смерть": ["смерть", "коса", "освобождающий", "преображение", "дитя великих перемен", "новь"],
    "Умеренность": ["умеренность", "воздержание", "воздержанность", "равновесие", "время", "алхимия", "человеческий гений", "смешивающий", "интеграция", "дочь примирения", "искусство", "ограничение"],
    "Дьявол": ["дьявол", "тифон", "абраксас", "бросающий вызов", "рогатый бог", "обуславливание", "повелитель врат сущего"],
    "Башня": ["башня", "разрушенная башня", "падающая башня", "богадельня", "дом божий", "повелитель всемогущего воинства", "изменяющий", "разрушение", "молния", "башня разрушаемая громом", "башня разрушаемая молнией"],
    "Звезда": ["звезда", "звёзды", "звезда магов", "страсть", "мечтатель", "молчание", "дочь небосвода"],
    "Луна": ["луна", "сумерки", "воображающий", "повелитель прилива и отлива", "прошлые жизни", "тайные враги"],
    "Солнце": ["солнце", "свет", "яркий свет", "тепло", "повелитель мирового огня", "защитник", "невинность"],
    "Суд": ["суд", "страшный суд", "космический будильник", "воскрешение", "воскрешение мертвых", "пробуждение мертвых", "эон", "возобновляющий", "дух первозданного огня", "за пределами иллюзий"],
    "Мир": ["мир", "корона магов", "универсум", "вселенная", "советник", "шаман", "одиночество", "великий в ночи времени"],
    # Младшие Арканы — Жезлы
    "Туз Жезлов": ["туз жезлов", "1 жезлов", "один жезл", "туз булав", "туз крестов", "туз палок", "туз посохов", "туз скипетров"],
    "Двойка Жезлов": ["двойка жезлов", "2 жезлов", "два жезла", "две палки", "двойка булав", "двойка крестов", "двойка посохов", "двойка скипетров"],
    "Тройка Жезлов": ["тройка жезлов", "3 жезлов", "три жезла", "три посоха", "тройка булав", "тройка крестов", "тройка палок", "тройка скипетров"],
    "Четверка Жезлов": ["четверка жезлов", "4 жезлов", "четыре жезла", "четыре посоха", "четверка булав", "четверка крестов", "четверка палок", "четверка скипетров"],
    "Пятерка Жезлов": ["пятерка жезлов", "5 жезлов", "пять жезлов", "пять посохов", "пятерка булав", "пятерка крестов", "пятерка палок", "пятерка скипетров"],
    "Шестерка Жезлов": ["шестерка жезлов", "6 жезлов", "шесть жезлов", "шесть посохов", "шестерка булав", "шестерка крестов", "шестерка палок", "шестерка скипетров"],
    "Семерка Жезлов": ["семерка жезлов", "7 жезлов", "семь жезлов", "семь посохов", "семерка булав", "семерка крестов", "семерка палок", "семерка скипетров"],
    "Восьмерка Жезлов": ["восьмерка жезлов", "8 жезлов", "восемь жезлов", "восемь посохов", "восьмерка булав", "восьмерка крестов", "восьмерка палок", "восьмерка скипетров"],
    "Девятка Жезлов": ["девятка жезлов", "9 жезлов", "девять жезлов", "девять посохов", "девятка булав", "девятка крестов", "девятка палок", "девятка скипетров"],
    "Десятка Жезлов": ["десятка жезлов", "10 жезлов", "десять жезлов", "десять посохов", "десятка булав", "десятка крестов", "десятка палок", "десятка скипетров"],
    "Паж Жезлов": ["паж жезлов", "паж булав", "паж крестов", "паж палок", "паж посохов", "паж скипетров"],
    "Рыцарь Жезлов": ["рыцарь жезлов", "рыцарь булав", "рыцарь крестов", "рыцарь палок", "рыцарь посохов", "рыцарь скипетров"],
    "Королева Жезлов": ["королева жезлов", "королева булав", "королева крестов", "королева палок", "королева посохов", "королева скипетров"],
    "Король Жезлов": ["король жезлов", "король булав", "король крестов", "король палок", "король посохов", "король скипетров"],
    # Младшие Арканы — Мечи
    "Туз Мечей": ["туз мечей", "1 мечей", "один меч", "туз сабель", "туз копий"],
    "Двойка Мечей": ["двойка мечей", "2 мечей", "два меча", "двойка сабель", "двойка копий"],
    "Тройка Мечей": ["тройка мечей", "3 мечей", "три меча", "тройка сабель", "тройка копий"],
    "Четверка Мечей": ["четверка мечей", "4 мечей", "четыре меча", "четверка сабель", "четверка копий"],
    "Пятерка Мечей": ["пятерка мечей", "5 мечей", "пять мечей", "пятерка сабель", "пятерка копий"],
    "Шестёрка Мечей": ["шестерка мечей", "6 мечей", "шесть мечей", "шестерка сабель", "шестерка копий"],
    "Семерка Мечей": ["семерка мечей", "7 мечей", "семь мечей", "семерка сабель", "семерка копий"],
    "Восьмерка Мечей": ["восьмерка мечей", "8 мечей", "восемь мечей", "восьмерка сабель", "восьмерка копий"],
    "Девятка Мечей": ["девятка мечей", "9 мечей", "девять мечей", "девятка сабель", "девятка копий"],
    "Десятка Мечей": ["десятка мечей", "10 мечей", "десять мечей", "десятка сабель", "десятка копий"],
    "Паж Мечей": ["паж мечей", "паж сабель", "паж копий"],
    "Рыцарь мечей": ["рыцарь мечей", "рыцарь сабель", "рыцарь копий"],
    "Королева мечей": ["королева мечей", "королева сабель", "королева копий"],
    "Король мечей": ["король мечей", "король сабель", "король копий"],
    # Младшие Арканы — Кубки
    "Туз Кубков": ["туз кубков", "1 кубков", "один кубок", "туз чаш", "туз чашек"],
    "Двойка Кубков": ["двойка кубков", "2 кубков", "два кубка", "двойка чаш", "двойка чашек"],
    "Тройка Кубков": ["тройка кубков", "3 кубков", "три кубка", "тройка чаш", "тройка чашек"],
    "Четверка Кубков": ["четверка кубков", "4 кубков", "четыре кубка", "четверка чаш", "четверка чашек"],
    "Пятерка Кубков": ["пятерка кубков", "5 кубков", "пять кубков", "пятерка чаш", "пятерка чашек"],
    "Шестерка Кубков": ["шестерка кубков", "6 кубков", "шесть кубков", "шестерка чаш", "шестерка чашек"],
    "Семерка Кубков": ["семерка кубков", "7 кубков", "семь кубков", "семерка чаш", "семерка чашек"],
    "Восьмерка Кубков": ["восьмерка кубков", "8 кубков", "восемь кубков", "восьмерка чаш", "восьмерка чашек"],
    "Девятка Кубков": ["девятка кубков", "9 кубков", "девять кубков", "девятка чаш", "девятка чашек"],
    "Десятка Кубков": ["десятка кубков", "10 кубков", "десять кубков", "десятка чаш", "десятка чашек"],
    "Паж Кубков": ["паж кубков", "паж чаш", "паж чашек"],
    "Рыцарь Кубков": ["рыцарь кубков", "рыцарь чаш", "рыцарь чашек"],
    "Королева Кубков": ["королева кубков", "королева чаш", "королева чашек"],
    "Король Кубков": ["король кубков", "король чаш", "король чашек"],
    # Младшие Арканы — Пентакли
    "Туз Пентаклей": ["туз пентаклей", "1 пентаклей", "один пентакль", "туз монет", "туз дисков", "туз динариев"],
    "Двойка Пентаклей": ["двойка пентаклей", "2 пентаклей", "два пентакля", "двойка монет", "двойка дисков", "двойка динариев"],
    "Тройка Пентаклей": ["тройка пентаклей", "3 пентаклей", "три пентакля", "тройка монет", "тройка дисков", "тройка динариев"],
    "Четверка Пентаклей": ["четверка пентаклей", "4 пентаклей", "четыре пентакля", "четверка монет", "четверка дисков", "четверка динариев"],
    "Пятерка Пентаклей": ["пятерка пентаклей", "5 пентаклей", "пять пентаклей", "пятерка монет", "пятерка дисков", "пятерка динариев"],
    "Шестерка Пентаклей": ["шестерка пентаклей", "6 пентаклей", "шесть пентаклей", "шестерка монет", "шестерка дисков", "шестерка динариев"],
    "Семерка Пентаклей": ["семерка пентаклей", "7 пентаклей", "семь пентаклей", "семерка монет", "семерка дисков", "семерка динариев"],
    "Восьмерка Пентаклей": ["восьмерка пентаклей", "8 пентаклей", "восемь пентаклей", "восьмерка монет", "восьмерка дисков", "восьмерка динариев"],
    "Девятка Пентаклей": ["девятка пентаклей", "9 пентаклей", "девять пентаклей", "девятка монет", "девятка дисков", "девятка динариев"],
    "Десятка Пентаклей": ["десятка пентаклей", "10 пентаклей", "десять пентаклей", "десятка монет", "десятка дисков", "десятка динариев"],
    "Паж Пентаклей": ["паж пентаклей", "паж монет", "паж дисков", "паж динариев"],
    "Рыцарь Пентаклей": ["рыцарь пентаклей", "рыцарь монет", "рыцарь дисков", "рыцарь динариев"],
    "Королева Пентаклей": ["королева пентаклей", "королева монет", "королева дисков", "королева динариев"],
    "Король Пентаклей": ["король пентаклей", "король монет", "король дисков", "король динариев"]

}

spheres_dict = {
    "Кратко": ["кратко", "коротко", "вкратце"],
    "Общее": ["общее", "общий", "в общем"],
    "Перевернутое": ["перевернутое", "перевернутая", "перевернутый"],
    "Совет": ["совет", "рекомендация"],
    "Карта дня": ["карта дня", "дневная карта"],
    "Финансы": ["финансы", "деньги", "богатство"],
    "Любовь": ["любовь", "отношения", "романтика", "любовные отношения", "любви"],
    "Здоровье": ["здоровье", "самочувствие", "болезнь", "физическое состояние"],
    "Личность": ["личность", "характер", "черты характера", "человек"],
    "Работа": ["работа", "карьера", "профессия", "труд", "бизнес", "работе"]
}


def normalize_card_name(card_text):
    card_text = card_text.lower()
    for card, aliases in cards_dict.items():
        if any(alias in card_text for alias in aliases):
            return card
    return None

def normalize_sphere_name(sphere_text):
    sphere_text = sphere_text.lower()
    for sphere, aliases in spheres_dict.items():
        if any(alias in sphere_text for alias in aliases):
            return sphere
    return None

# Функция обработки текстового сообщения для поиска карты и сферы
def extract_card_and_sphere(text):
    card = None
    sphere = None

    # Пробуем найти карту
    for card_name, aliases in cards_dict.items():
        for alias in aliases:
            if re.search(rf"\b{alias}\b", text, re.IGNORECASE):
                card = card_name
                break
        if card:
            break

    # Пробуем найти сферу
    for sphere_name, aliases in spheres_dict.items():
        for alias in aliases:
            if re.search(rf"\b{alias}\b", text, re.IGNORECASE):
                sphere = sphere_name
                break
        if sphere:
            break

    return card, sphere

@bot.message_handler(commands=['help'])
def help_command(message):
    print("Команда /help вызвана.")
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    card_button = types.KeyboardButton("🔮 Узнать значение карты в системе Уэйта")
    markup.add(card_button)
    bot.send_message(message.chat.id, "Я могу помочь тебе с трактовкой карт Таро. Используй команды:\n/start — Начало работы\n/help — Инструкция\nНажми на кнопку ниже, чтобы узнать значение карты.", reply_markup=markup)

@bot.message_handler(func=lambda message: message.text == "🔮 Узнать значение карты в системе Уэйта")
def choose_sphere(message):
    print("Выбрана кнопка 'Узнать значение карты'.")
    markup = types.InlineKeyboardMarkup(row_width=2)
    spheres = ['Кратко', 'Общее', 'Перевернутое', 'Совет', 'Карта дня', 'Финансы', 'Любовь', 'Здоровье', 'Личность', 'Работа']
    buttons = [types.InlineKeyboardButton(f"{sphere} {emoji}", callback_data=f"sphere_{sphere}") for sphere, emoji in zip(spheres, ['👁', '🌞', '🌚', '🤫', '📅', '💸', ' и отношения❤️', '🧘‍♀️', '🧝‍♀️', '👩‍💻'])]
    markup.add(*buttons)
    bot.send_message(message.chat.id, "Выбери сферу:", reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data.startswith("sphere_"))
def choose_card_category(call):
    sphere = call.data.split("_")[1]
    user_selection[call.message.chat.id] = {'sphere': sphere}
    print(f"Сфера '{sphere}' выбрана.")

    markup = types.InlineKeyboardMarkup(row_width=2)
    btn1 = types.InlineKeyboardButton("Старшие арканы 💫", callback_data="major_arcana")
    btn2 = types.InlineKeyboardButton("Жезлы 🌳", callback_data="wands")
    btn3 = types.InlineKeyboardButton("Кубки 🍷", callback_data="cups")
    btn4 = types.InlineKeyboardButton("Мечи ⚔️", callback_data="swords")
    btn5 = types.InlineKeyboardButton("Пентакли 💵", callback_data="pentacles")
    markup.add(btn1, btn2, btn3, btn4, btn5)

    bot.send_message(call.message.chat.id, f"Сфера '{sphere}' выбрана. Теперь выбери категорию карт:", reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data in ["major_arcana", "wands", "cups", "swords", "pentacles"])
def choose_specific_card(call):
    category = call.data
    markup = types.InlineKeyboardMarkup(row_width=3)

    if category == "major_arcana":
        cards = ["Шут", "Маг", "Жрица", "Императрица", "Император", "Иерофант ", "Влюбленные", "Колесница", "Сила", "Отшельник", "Колесо Фортуны", "Справедливость", "Повешенный", "Смерть", "Умеренность", "Дьявол", "Башня", "Звезда", "Луна", "Солнце", "Суд", "Мир"]
    elif category == "wands":
        cards = ["Туз Жезлов", "Двойка Жезлов", "Тройка Жезлов", "Четверка Жезлов", "Пятерка Жезлов", "Шестерка Жезлов", "Семерка Жезлов", "Восьмерка Жезлов", "Девятка Жезлов", "Десятка Жезлов", "Паж Жезлов", "Рыцарь Жезлов", "Королева Жезлов", "Король Жезлов"]
    elif category == "cups":
        cards = ["Туз Кубков", "Двойка Кубков", "Тройка Кубков", "Четверка Кубков", "Пятерка Кубков", "Шестерка Кубков", "Семерка Кубков", "Восьмерка Кубков", "Девятка Кубков", "Десятка Кубков", "Паж Кубков", "Рыцарь Кубков", "Королева Кубков", "Король Кубков"]
    elif category == "swords":
        cards = ["Туз Мечей", "Двойка Мечей", "Тройка Мечей", "Четверка Мечей", "Пятерка Мечей", "Шестёрка Мечей", "Семерка Мечей", "Восьмерка Мечей", "Девятка Мечей", "Десятка Мечей", "Паж Мечей", "Рыцарь мечей", "Королева мечей", "Король мечей"]
    elif category == "pentacles":
        cards = ["Туз Пентаклей", "Двойка Пентаклей", "Тройка Пентаклей", "Четверка Пентаклей", "Пятерка Пентаклей", "Шестерка Пентаклей", "Семерка Пентаклей", "Восьмерка Пентаклей", "Девятка Пентаклей", "Десятка Пентаклей", "Паж Пентаклей", "Рыцарь Пентаклей", "Королева Пентаклей", "Король Пентаклей"]

    buttons = [types.InlineKeyboardButton(card, callback_data=f"card_{card}") for card in cards]
    markup.add(*buttons)

    bot.send_message(call.message.chat.id, "Выбери карту:", reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data.startswith("card_"))
def show_card_meaning(call):
    card = call.data.split("_")[1]
    chat_id = call.message.chat.id
    sphere = user_selection[chat_id].get('sphere')

    # Получаем значение карты из DataFrame
    card_data = meaning_df[meaning_df['Карта'] == card]

    if not card_data.empty:
        meaning = card_data[sphere].values[0]
        bot.send_message(call.message.chat.id, f"**Значение карты:** *{card}*\n**Сфера:** *{sphere}*\n\n{meaning}", parse_mode='Markdown')
    else:
        bot.send_message(call.message.chat.id, "Извините, значение карты не найдено.")



@bot.message_handler(func=lambda message: True)
def handle_free_text(message):
    chat_id = message.chat.id
    text = message.text

    card, sphere = extract_card_and_sphere(text)

    if card:
        card_data = meaning_df[meaning_df['Карта'] == card]

        if not card_data.empty:
            if sphere:
                # Если карта и сфера найдены, выводим значение карты по сфере
                meaning = card_data[sphere].values[0]
                bot.send_message(chat_id, f"**Значение карты:** *{card}*\n**Сфера:** *{sphere}*\n\n{meaning}", parse_mode='Markdown')
            else:
                # Если сфера не указана, выводим все значения карты по всем сферам
                all_meanings = []
                for col in meaning_df.columns[1:]:  # Пропускаем столбец с картой
                    meaning = card_data[col].values[0]
                    all_meanings.append(f"**Сфера:** *{col}*\n{meaning}")
                bot.send_message(chat_id, f"**Значение карты:** *{card}*\n\n" + "\n\n".join(all_meanings), parse_mode='Markdown')
        else:
            bot.send_message(chat_id, "Извините, значение карты не найдено.")
    else:
        bot.send_message(chat_id, "Не удалось распознать карту. Попробуйте снова.")

bot.polling(none_stop=True)

Файл успешно загружен.


2024-10-19 13:33:19,133 (__init__.py:1241 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message is too long"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message is too long
2024-10-19 13:33:19,138 (__init__.py:1243 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1235, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 150, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 93, in run
    task(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 8945, in _run_middlewares_and_handler
    result = hand

Команда /help вызвана.
Выбрана кнопка 'Узнать значение карты'.
Сфера 'Совет' выбрана.
Сфера 'Кратко' выбрана.


In [ ]:
print(meaning_df.iloc [27])

Карта                                              Шестёрка Мечей
Кратко                        Путешествие, перемены, освобождение
Общее           Шестёрка Мечей символизирует путешествие, пере...
Перевернутое    В перевернутом положении Шестёрка Мечей может ...
Совет           Карта советует вам быть открытыми к переменам ...
Карта дня       В качестве карты дня Шестёрка Мечей предупрежд...
Финансы         В финансовом плане карта может указывать на не...
Любовь          В отношениях Шестёрка Мечей может означать нео...
Здоровье        Карта может указывать на необходимость обратит...
Личность        Человек, олицетворяемый Шестёркой Мечей, часто...
Работа          На работе Шестёрка Мечей может указывать на не...
Name: 27, dtype: object
